In [8]:
pip install -qU pinecone-client pinecone-text pinecone-notebooks langchain-community langchain-huggingface langchain-chroma

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 91.7 MB/s eta 0:00:00


In [9]:
!pip install -qU nltk
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Insert you Pinecone api_key
api_key=input() # delete the output for safety

In [11]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [12]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
## initialize the Pinecone client
pc=Pinecone(api_key=api_key)

# create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
      name=index_name,
      dimension=384, # dimension of dense vector
      metric='dotproduct',  ## sparse values supportted only for dotproduct
      spec=ServerlessSpec(cloud='aws',region="us-east-1")
    )

In [14]:
index=pc.Index(index_name)
index

In [15]:
## vector embedding and spare matrix
import os

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN") or input()

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="All-MiniLM-L6-v2")
embeddings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HuggingFaceEmbeddings(model_name='All-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [16]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
sentences=[
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orelans",
]

## tfidf values on these sentence
bm25_encoder.fit(sentences)
# bm25_encoder.fit(df['plot_synopsis'] + df['title'])
## store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Ecnoder object

bm25_encoder = BM25Encoder().load("bm25_values.json")

  0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='All-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x7adddfa5fcd0>, index=<pinecone.data.index.Index object at 0x7adec9950cd0>)

In [19]:
retriever.add_texts(
    ["In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orelans",])

# retriever.add_texts(df['plot_synopsis'].tolist())

  0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
retriever.invoke("What city did I visit last")

[Document(metadata={'score': 0.255652815}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.235293239}, page_content='In 2023, I visited Paris'),
 Document(metadata={'score': 0.210283875}, page_content='In 2021, I visited New Orelans')]

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("cryptexcode/mpst-movie-plot-synopses-with-tags")

print("Path to dataset files:", path)

100%|██████████| 28.8M/28.8M [00:00<00:00, 56.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/cryptexcode/mpst-movie-plot-synopses-with-tags/versions/1


In [4]:
import pandas as pd
# Assuming the dataset is a CSV file named 'mpst_full_data.csv'
try:
    df = pd.read_csv(path + '/mpst_full_data.csv')
    print(df.head())
except FileNotFoundError:
    print(f"Error: File not found at {path + '/mpst_full_data.csv'}. Check the path or filename.")
except Exception as e:
    print(f"An error occurred: {e}")

     imdb_id                                          title  \
0  tt0057603                        I tre volti della paura   
1  tt1733125  Dungeons & Dragons: The Book of Vile Darkness   
2  tt0033045                     The Shop Around the Corner   
3  tt0113862                             Mr. Holland's Opus   
4  tt0086250                                       Scarface   

                                       plot_synopsis  \
0  Note: this synopsis is for the orginal Italian...   
1  Two thousand years ago, Nhagruul the Foul, a s...   
2  Matuschek's, a gift store in Budapest, is the ...   
3  Glenn Holland, not a morning person by anyone'...   
4  In May 1980, a Cuban man named Tony Montana (A...   

                                                tags  split synopsis_source  
0          cult, horror, gothic, murder, atmospheric  train            imdb  
1                                           violence  train            imdb  
2                                           romant